In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/kaggle/input/iris-flower-dataset/IRIS.csv')

In [3]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
df['species'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [5]:
df['species'].replace({'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2}, inplace=True)

/tmp/ipykernel_36/2485095.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['species'].replace({'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2}, inplace=True)
/tmp/ipykernel_36/2485095.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['species'].replace({'Iris-setosa':0, 'Iris-ve

In [6]:
def preprocess(x,y):
    enc = OneHotEncoder()
    y_out = enc.fit_transform(y.values.reshape(-1,1))
    scaler = StandardScaler()
    x_out = scaler.fit_transform(x)
    return x_out,y_out.toarray()
    

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,accuracy_score
from sklearn.preprocessing import OneHotEncoder

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(["species"],axis=1),df["species"])

In [11]:
X_train.shape, X_test.shape

((112, 4), (38, 4))

In [12]:
xin,yin = preprocess(X_train,y_train)

In [13]:
y_train.head()

122    2
79     1
63     1
89     1
6      0
Name: species, dtype: int64

In [15]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchmetrics
from torch.utils.data import TensorDataset, DataLoader

In [17]:
class ClassificationModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = torch.nn.Sequential(
            torch.nn.Linear(4, 3),
            torch.nn.Softmax(dim=1)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = ClassificationModel().to(device)
print(model)

ClassificationModel(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=3, bias=True)
    (1): Softmax(dim=1)
  )
)


In [18]:
xt,yt = preprocess(X_test,y_test)

In [19]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [20]:
def train(model,epoch,loss_fn, optimizer,xin,yin):
    model.train()
    p = np.random.permutation(len(xin))
    tempx = xin[p]
    tempy = yin[p]
    pred = model(torch.from_numpy(tempx).to(torch.float32))
    loss = loss_fn(pred, torch.from_numpy(tempy).to(torch.float32))

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    loss, current = loss.item(), epoch
    print(f"loss: {loss:>7f}  [{current:>5d}")

In [21]:
def test(model, loss_fn,xt,yt):
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        X, y = torch.from_numpy(xt).to(torch.float32), torch.from_numpy(yt).to(torch.float32)
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
    print(f"Test Error: Avg loss: {test_loss:>8f} \n")

In [22]:
epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model,t, loss_fn, optimizer,xin,yin)
    test(model, loss_fn,xt,yt)
print("Done!")

Epoch 1
-------------------------------
loss: 1.224760  [    0
Test Error: Avg loss: 1.159721 

Epoch 2
-------------------------------
loss: 1.158162  [    1
Test Error: Avg loss: 1.098863 

Epoch 3
-------------------------------
loss: 1.092422  [    2
Test Error: Avg loss: 1.036591 

Epoch 4
-------------------------------
loss: 1.028641  [    3
Test Error: Avg loss: 0.970120 

Epoch 5
-------------------------------
loss: 0.962573  [    4
Test Error: Avg loss: 0.900195 

Epoch 6
-------------------------------
loss: 0.893138  [    5
Test Error: Avg loss: 0.841619 

Epoch 7
-------------------------------
loss: 0.832268  [    6
Test Error: Avg loss: 0.804467 

Epoch 8
-------------------------------
loss: 0.791179  [    7
Test Error: Avg loss: 0.783614 

Epoch 9
-------------------------------
loss: 0.767783  [    8
Test Error: Avg loss: 0.770918 

Epoch 10
-------------------------------
loss: 0.753491  [    9
Test Error: Avg loss: 0.762227 

Epoch 11
------------------------------

In [23]:
def print_report(y_true,y_pred):
    print(classification_report(y_true,y_pred))
    print(f"Accuracy:- {accuracy_score(y_true,y_pred)}")

In [24]:
with torch.no_grad():
    x = torch.from_numpy(xt).to(device).to(torch.float32)
    pred = model(x)
    print_report(yt,(pred>0.5))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.92      0.96        13
           2       0.92      1.00      0.96        12

   micro avg       0.97      0.97      0.97        38
   macro avg       0.97      0.97      0.97        38
weighted avg       0.98      0.97      0.97        38
 samples avg       0.97      0.97      0.97        38

Accuracy:- 0.9736842105263158
